# Clustering Models

In [1]:
import numpy as np
import pandas as pd
import sklearn
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import dtale
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import *
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D

pd.set_option('display.max_columns', 300)

In [13]:
file = open('dataframe.p', 'rb')
df1 = pickle.load(file)
file.close()
file = open('data.p', 'rb')
df2 = pickle.load(file)
file.close()

In [14]:
# Set df1 as article text data
df1 = df1.iloc[:, :5]

In [15]:
model_df = df2

In [16]:
X_model = model_df.drop(columns=['source', 'total_sentences'])
y_model = model_df['source']

## Pre-Model Scaling and PCA

In [78]:
# Scale data for PCA
scaler = RobustScaler()
X_model = scaler.fit_transform(X_model)

In [79]:
# Find lowest n-components with explained variance > 95%
for n in range(2, 10):
    pca_ = PCA(n_components=n)
    pca_.fit_transform(X_model)
    print(f'{n} Components: {np.sum(pca_.explained_variance_ratio_)}')

2 Components: 0.6988869143328772
3 Components: 0.8814884095785016
4 Components: 0.92902819993247
5 Components: 0.9461266570171747
6 Components: 0.9599591787469712
7 Components: 0.9693773194268571
8 Components: 0.9784806907902082
9 Components: 0.9842831777377682


In [80]:
# Fit and transform data to 6 principal components
pca_model = PCA(n_components=6)
X_model = pca_model.fit_transform(X_model)

## Models

### Hierarchical and K-Means
- Hierarchical Clustering: highest silhouette score - 0.445 (n=3)
- K-Means Clustering: highest silhouette score - 0.445 (n=3)

In [237]:
# Find best n-clusters for AggClustering and K-Means
for n in range(2, 10):
    hc = AgglomerativeClustering(n_clusters=n, affinity='euclidean', linkage='ward')
    kc = KMeans(n_clusters=n)
    y_hc = hc.fit(X_model)
    y_kc = kc.fit(X_model)
    print(f'---{n} Clusters---')
    print('Hierarchical Clustering:', sklearn.metrics.silhouette_score(X_model, y_hc.labels_))
    print('K-Means Clustering:', sklearn.metrics.silhouette_score(X_model, y_kc.labels_))

---2 Clusters---
Hierarchical Clustering: 0.41739722999722556
K-Means Clustering: 0.3956877921811657
---3 Clusters---
Hierarchical Clustering: 0.4453342206250786
K-Means Clustering: 0.44598852185215504
---4 Clusters---
Hierarchical Clustering: 0.3421939476719469
K-Means Clustering: 0.38936936460288524
---5 Clusters---
Hierarchical Clustering: 0.3448098976763754
K-Means Clustering: 0.38504984442759516
---6 Clusters---
Hierarchical Clustering: 0.302750759077561
K-Means Clustering: 0.3597639413472225
---7 Clusters---
Hierarchical Clustering: 0.2579201695638342
K-Means Clustering: 0.31265374363688186
---8 Clusters---
Hierarchical Clustering: 0.259082697881445
K-Means Clustering: 0.30823618845247647
---9 Clusters---
Hierarchical Clustering: 0.2506100332865101
K-Means Clustering: 0.3087184318431184


### DBSCAN and OPTICS
- DBSCAN: highest silhouette score - 0.523 (n=2, eps=1.9)
- OPTICS: highest silhouette score - 0.461 (n=2, min_samples=57)

In [233]:
# DBSCAN
for n in np.arange(1.5, 2.3, 0.1):
    dbc = DBSCAN(eps=n)
    y_dbc = dbc.fit(X_model)
    a = sklearn.metrics.silhouette_score(X_model, y_dbc.labels_)
    print(f'Sil. Score (eps={round(n, 1)}): {a}')

Sil. Score (eps=1.5): 0.4088022701568945
Sil. Score (eps=1.6): 0.4752118472495868
Sil. Score (eps=1.7): 0.47621087752873714
Sil. Score (eps=1.8): 0.493449468699244
Sil. Score (eps=1.9): 0.5231225330914477
Sil. Score (eps=2.0): 0.5231225330914477
Sil. Score (eps=2.1): 0.30223759325537974
Sil. Score (eps=2.2): 0.3666400078126211


In [235]:
# OPTICS
for n in range(55, 65, 1):
    opc = OPTICS(min_samples=n, metric='correlation')
    y_opc = opc.fit(X_model)
    a = sklearn.metrics.silhouette_score(X_model, y_opc.labels_)
    print(f'Sil. Score (min_samples={n}): {a}')

Sil. Score (min_samples=55): 0.3396842390727309
Sil. Score (min_samples=56): 0.3470341153311355
Sil. Score (min_samples=57): 0.3533743662024179
Sil. Score (min_samples=58): 0.22551165740410267
Sil. Score (min_samples=59): 0.34916620280664645
Sil. Score (min_samples=60): 0.3500241313641878
Sil. Score (min_samples=61): 0.23040803136171034
Sil. Score (min_samples=62): 0.21894397976989555
Sil. Score (min_samples=63): 0.3376536937527383
Sil. Score (min_samples=64): 0.3450754196445452


### Mean Shift and Spectral Clustering
- Mean Shift: highest silhouette score - 0.590 (n=2, bandwidth=3.4)
- Spectral Clustering: highest silhouette score - 0.641 (n=2)

In [223]:
# Mean Shift 
for n in np.arange(3.3, 3.9, 0.1):
    msc = MeanShift(bandwidth=n)
    y_msc = msc.fit(X_model)
    print(f'Sil. Score (bw={round(n, 1)}): {sklearn.metrics.silhouette_score(X_model, y_msc.labels_)}')

Sil. Score (bw=3.3): 0.5663253565559246
Sil. Score (bw=3.4): 0.590106401458918
Sil. Score (bw=3.5): 0.590106401458918
Sil. Score (bw=3.6): 0.590106401458918
Sil. Score (bw=3.7): 0.5771404868710135
Sil. Score (bw=3.8): 0.5752226448702275
Sil. Score (bw=3.9): 0.5771404868710135


In [226]:
# Spectral Clustering
for n in range(2, 8):
    scc = SpectralClustering(n_clusters=n)
    y_scc = scc.fit(X_model)
    print(f'Sil. Score ({n} clusters): {sklearn.metrics.silhouette_score(X_model, y_scc.labels_)}')

Sil. Score (2 clusters): 0.6410672191821027
Sil. Score (3 clusters): 0.46726725162531474
Sil. Score (4 clusters): 0.4506105083954233
Sil. Score (5 clusters): 0.4515478423051401
Sil. Score (6 clusters): 0.40962889597355095
Sil. Score (7 clusters): 0.36235355183434953


### Affinity Propagation

In [225]:
# Affinity Propagation
for n in np.arange(0.86, 0.92, 0.01):
    apc = AffinityPropagation(damping=n)
    y_apc = apc.fit(model_df.drop(columns=['source', 'total_sentences']))
    print(f'Sil. Score (damping={round(n, 2)}): {sklearn.metrics.silhouette_score(X_model, y_apc.labels_)}')

Sil. Score (damping=0.86): 0.036640736438425116
Sil. Score (damping=0.87): 0.03341753961052408
Sil. Score (damping=0.88): 0.03413792355621138
Sil. Score (damping=0.89): 0.036640736438425116
Sil. Score (damping=0.9): 0.036640736438425116
Sil. Score (damping=0.91): 0.03203377500144594
Sil. Score (damping=0.92): 0.03413792355621138


### Notes
- DBSCAN had 2 clusters: 1 labeled cluster, 1 outliers cluster
- OPTICS had 1 cluster
- Mean Shift had 2 clusters: Cluster 0 had 1297 obs, Cluster 1 had 44 obs (43 were CCTV articles)
- Spectral Clustering had 2 clusters: Cluster 0 had 1319 obs, Cluster 1 had 22 obs (all CCTV)
- Affinity Propagation had 44 clusters

In [92]:
# Top clustering models
hc_model = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
kc_model = KMeans(n_clusters=3)
dbc_model = DBSCAN(eps=1.9)
opc_model = OPTICS(min_samples=57)
msc_model = MeanShift(bandwidth=3.4)
scc_model = SpectralClustering(n_clusters=2)
apc_model = AffinityPropagation(damping=0.87)

# Labels from models
y_hc = hc_model.fit(X_model).labels_.reshape(1341,1)
y_kc = kc_model.fit(X_model).labels_.reshape(1341,1)
y_dbc = dbc_model.fit(X_model).labels_.reshape(1341,1)
y_opc = opc_model.fit(X_model).labels_.reshape(1341,1)
y_msc = msc_model.fit(X_model).labels_.reshape(1341,1)
y_scc = scc_model.fit(X_model).labels_.reshape(1341,1)
y_apc = apc_model.fit(X_model).labels_.reshape(1341,1)

In [93]:
# Determine number of clusters per model
labels = [y_hc, y_kc, y_dbc, y_opc, y_msc, y_scc, y_apc]
models = ['Hierarchical', 'K-Means', 'DBSCAN', 'OPTICS', 'Mean Shift', 'Spectral', 'Aff. Prop.']
mod_lab = list(zip(models, labels))
for m, l in mod_lab:
    print(f'{m} - Number of Labels: {len(set(l.reshape(1341,)))}')

Hierarchical - Number of Labels: 3
K-Means - Number of Labels: 3
DBSCAN - Number of Labels: 2
OPTICS - Number of Labels: 1
Mean Shift - Number of Labels: 2
Spectral - Number of Labels: 2
Aff. Prop. - Number of Labels: 44


## Model Visualizations

In [94]:
# Create a DataFrame for data visualizations
data_col = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6']

mod_lab_df = pd.DataFrame(np.concatenate((y_hc, y_kc, y_dbc, y_opc, y_msc, y_scc, y_apc, X_model), axis=1))
mod_lab_df.columns = models + data_col

viz_df = pd.concat((mod_lab_df, model_df, df1['headline'], df1['date']), axis=1)

viz_df['headline'] = viz_df['headline'].map(lambda x: ' '.join(x.split()[:10]))
viz_df['date'] = viz_df['date'].map(lambda x: x.date())

viz_df['Hierarchical'] = viz_df['Hierarchical'].map(lambda x: round(x)).astype(str)
viz_df['K-Means'] = viz_df['K-Means'].map(lambda x: round(x)).astype(str)
viz_df['DBSCAN'] = viz_df['DBSCAN'].map(lambda x: round(x)).astype(str)
viz_df['OPTICS'] = viz_df['OPTICS'].map(lambda x: round(x)).astype(str)
viz_df['Mean Shift'] = viz_df['Mean Shift'].map(lambda x: round(x)).astype(str)
viz_df['Spectral'] = viz_df['Spectral'].map(lambda x: round(x)).astype(str)
viz_df['Aff. Prop.'] = viz_df['Aff. Prop.'].map(lambda x: round(x)).astype(str)

In [95]:
# file = open('viz_df.p', 'wb')      
# pickle.dump(viz_df, file)
# file.close()

In [96]:
file = open('viz_df.p', 'rb')      
viz_df = pickle.load(file)
file.close()

In [98]:
# Find extremes
group1 = viz_df['total_sentences']
group2 = viz_df['date']

a = viz_df.loc[(group1 > 5) & (group2 > pd.datetime(2019, 9, 1).date())]
group3 = (a['gov'] + a['poli'] + a['econ'] + a['protest'])
a.loc[(group3 == group3.min())|(group3 == group3.max())]

,Hierarchical,K-Means,DBSCAN,OPTICS,Mean Shift,Spectral,Aff. Prop.,pc1,pc2,pc3,pc4,pc5,pc6,source,protest,econ,poli,gov,protest_mention,econ_mention,poli_mention,gov_mention,total_sentences,w_protest,w_econ,w_gov,w_poli,hl_sent,protest_ratio,econ_ratio,poli_ratio,gov_ratio,headline,date
324,0,0,0,0,0,0,0,-0.053410,-1.710457,-0.321447,-1.407695,-0.100756,0.013109,SCMP,0.48,0.0,0.3,0.03,2,0,3,1,6,0.160000,0.0,0.005000,0.15,0.2960,0.333333,0.0,0.5,0.166667,"No backlash from Beijing, Malaysian PM Mahathi...",2019-10-08
1070,1,2,0,0,0,0,31,6.170092,3.083828,-1.577847,1.987518,0.818163,-0.117762,CCTV,3.27,0.0,0.0,3.68,4,0,0,5,9,1.453333,0.0,2.044444,0.00,0.8271,0.444444,0.0,0.0,0.555556,Carrie Lam: Violence is relentlessly destroyin...,2019-11-12


In [100]:
# Highest "strength of language" after 2019, 9, 1
df1.iloc[1070]['sentences'][:4]

['HKSAR Chief Executive Carrie Lam on Monday condemned violent protesters who are relentlessly destroying Hong Kong society and called for people to stay calm.She made the remarks at a press conference on Monday after the recent escalation in protests, giving a statement on what is said to be one of the worst days of violence since protests erupted in the city in June.',
 'Lam confirmed that one person had been injured after being shot by a police officer on Monday and another individual had been set on fire by protesters.',
 'Violence is not the solution and will only trigger more violence, said Lam, stressing that escalating violence will not make government yield to pressure.',
 'In response to a question raised by CGTN, Lam reiterated that it is Hong Kongs priority to end the violence and restore order.']

In [65]:
# Lowest "strength of language" after 2019, 9, 1
df1.iloc[324]['sentences'][:4]

['Despite calling for the resignation of Carrie Lam Cheng Yuet-ngor as Hong Kong’s chief executive, Malaysian Prime Minister Mahathir Mohamad said he did not receive any backlash from China.',
 'Carrie Lam Cheng Yuet-ngor Mahathir Mohamad Mahathir said he only commented on the issue because it was a question from a Hong Kong lawyer during an event.',
 'commented on the issue “I did not want to pass any opinion of what is happening there because I regard it as an internal matter.',
 'I was asked for my opinion as to what [Lam] should do, and my reply was that she is in a dilemma and she should resign.']

In [112]:
# Function to create 2D plot of principal components with highlighted areas of clusters
def two_d(mod, viz_df=viz_df, components=['pc1', 'pc3']):
    n_clusters = len(viz_df[mod].unique())
    c_label = list(viz_df[mod].unique())
    color_map = {'0': 'green', '1': 'blue', '2': 'red', '-1': 'fuchsia'}
    fig = px.scatter(viz_df,
                     x=components[0],
                     y=components[1],
                     symbol='source',
                     color=mod,
                     color_discrete_map=color_map,
                     opacity=0.8,
                     hover_name='headline',
                     hover_data=[mod, 'date'],
                    )
    
    fig.update_layout(legend_orientation="v")
    
    # Create cluster areas
    dicts = []
    for i in c_label:
        if i != '-1':
            a = dict(type="circle", xref="x", yref="y",
                     x0=min(viz_df.loc[viz_df[mod] == f'{i}' ][components[0]]),
                     y0=min(viz_df.loc[viz_df[mod] == f'{i}' ][components[1]]),
                     x1=max(viz_df.loc[viz_df[mod] == f'{i}' ][components[0]]),
                     y1=max(viz_df.loc[viz_df[mod] == f'{i}' ][components[1]]),
                     opacity=0.2, fillcolor=color_map[f'{i}'],
                     line_color=color_map[f'{i}'])
            dicts.append(a)    
    fig.update_layout(shapes=dicts)
    
    fig.update_layout(
        title={'text': f'{mod} (n={n_clusters}) in 2D (n-components=6)',
               'y':0.9,
               'x':0.5,
               'xanchor': 'center',
               'yanchor': 'top'},
        font=dict(family='Arial',
                  size=18,
                  color='#7f7f7f'))
    
    fig.update_layout(xaxis_title=f"Principal Component: {components[0][2]}",
                      yaxis_title=f"Principal Component: {components[1][2]}",
                      font=dict(family='Arial',
                                size=12,
                                color='#7f7f7f'))
    
    fig.show()

In [114]:
two_d('Spectral', components=['pc1', 'pc6'])

In [76]:
def three_d(mod, viz_df=viz_df, components=['pc1', 'pc4', 'pc5']):
    fig = px.scatter_3d(viz_df, x=components[0], y=components[1], z=components[2],
                        color=mod, symbol='source', opacity=0.8,
                        hover_name='headline', hover_data=[mod])
    fig.update_layout(showlegend=True)
    fig.show()

In [247]:
three_d('Spectral', components=['pc1', 'pc3', 'pc5'])

In [ ]:
px.density_contour(viz_df.loc[cond2|cond4], x='econ', y='gov', color='source')

In [249]:
# Function to group sources by model cluster
def source_group(mod):
    labels = sorted(viz_df[mod].unique())
    a = pd.DataFrame(viz_df.groupby('source')[mod].value_counts(normalize=True, sort=False))
    a = round(a.unstack() * 100, 1)
    a = np.where(a.isna(), 0, a)
    return labels, a

In [250]:
def h_bar(mod):
    sources = ['ABC (Australia)', 'CCTV', 'CNN', 'Reuters', 'SCMP']
    labels, ls = source_group(mod)
    x_data = ls
    y_data = sources
    top_labels = labels
    colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
              'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)']

    fig = go.Figure()

    for i in range(0, len(x_data[0])):
        for xd, yd in zip(x_data, y_data):
            fig.add_trace(go.Bar(
                x=[xd[i]], y=[yd],
                orientation='h',
                marker=dict(
                    color=colors[i],
                    line=dict(color='rgb(248, 248, 249)', width=1)
                )
            ))

    fig.update_layout(
        xaxis=dict(
            showgrid=False,
            showline=False,
            showticklabels=False,
            zeroline=False,
            domain=[0.15, 1]
        ),
        yaxis=dict(
            showgrid=False,
            showline=False,
            showticklabels=False,
            zeroline=False,
        ),
        barmode='stack',
        paper_bgcolor='rgb(248, 248, 255)',
        plot_bgcolor='rgb(248, 248, 255)',
        margin=dict(l=10, r=40, t=140, b=80),
        showlegend=False,
    )

    annotations = []

    for yd, xd in zip(y_data, x_data):
        # labeling the y-axis
        annotations.append(dict(xref='paper', yref='y',
                                x=0.14, y=yd,
                                xanchor='right',
                                text=str(yd),
                                font=dict(family='Arial', size=10,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False, align='right'))
        # labeling the first percentage of each bar (x_axis)
        annotations.append(dict(xref='x', yref='y',
                                x=xd[0] / 2, y=yd,
                                text=str(xd[0]) + '%',
                                font=dict(family='Arial', size=10,
                                          color='rgb(248, 248, 255)'),
                                showarrow=False))
        # labeling the first cluster of a model (on the top)
        if yd == y_data[-1]:
            annotations.append(dict(xref='x', yref='paper',
                                    x=xd[0] / 2, y=1.1,
                                    text=top_labels[0],
                                    font=dict(family='Arial', size=10,
                                              color='rgb(67, 67, 67)'),
                                    showarrow=False))
        space = xd[0]
        for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd,
                                    text=str(xd[i]) + '%',
                                    font=dict(family='Arial', size=10,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # Labeling clusters of model
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=10,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

    fig.update_layout(annotations=annotations)
    fig.update_layout(
        title={'text': f'{mod} Cluster Groups by Source',
               'y':0.9,
               'x':0.5,
               'xanchor': 'center',
               'yanchor': 'top'},
               font=dict(
                   family='Arial',
                   size=18,
                   color='#7f7f7f'))
    fig.show()

In [258]:
h_bar('K-Means')

1159